# Notebook 03: Feature Engineering

**Objetivo**: Crear features temporales, de rezago y estadísticas para modelado predictivo de urgencias.

## Estrategia:
1. Cargar datos de urgencias redefinidas (Definición A: Percentil 75)
2. Filtrar productos con predictibilidad Moderada/Alta
3. Crear features:
   - **Temporales**: semana, mes, trimestre, estacionalidad
   - **Lags**: rezagos de ventas (1, 2, 4, 8, 52 semanas)
   - **Rolling stats**: MA, std, min, max en ventanas móviles
   - **Urgency-specific**: días desde última urgencia, frecuencia
4. Preparar train/test split temporal
5. Guardar dataset listo para modelado

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Librerías cargadas")

## 1. Carga de Datos

In [ ]:
# Cargar urgencias redefinidas
df_urgencias = pd.read_csv('../data/simulated/urgencias_redefinidas.csv')
df_urgencias['date'] = pd.to_datetime(df_urgencias['date'])

# Cargar clasificación de productos
df_productos = pd.read_csv('../data/simulated/productos_predecibles.csv')

print(f"Datos cargados:")
print(f"  • Registros totales: {len(df_urgencias):,}")
print(f"  • Productos únicos: {df_urgencias['item_id'].nunique():,}")
print(f"  • Rango temporal: {df_urgencias['date'].min()} a {df_urgencias['date'].max()}")
print(f"\nDistribución de predictibilidad:")
print(df_productos['clasificacion'].value_counts())

In [ ]:
# Filtrar solo productos Predecibles y Moderados
productos_modelar = df_productos[
    df_productos['clasificacion'].isin(['Predecible', 'Moderado'])
]['item_id'].unique()

df = df_urgencias[df_urgencias['item_id'].isin(productos_modelar)].copy()
df = df.sort_values(['item_id', 'date']).reset_index(drop=True)

print(f"\n✓ Dataset filtrado:")
print(f"  • Productos a modelar: {len(productos_modelar):,}")
print(f"  • Registros: {len(df):,}")
print(f"  • Proporción urgencias: {df['is_urgent_a'].mean():.1%}")

## 2. Features Temporales

In [ ]:
# Extraer componentes temporales
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter
df['week_of_year'] = df['date'].dt.isocalendar().week
df['day_of_year'] = df['date'].dt.dayofyear

# Features cíclicas (sin/cos para capturar periodicidad)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df['week_sin'] = np.sin(2 * np.pi * df['week_of_year'] / 52)
df['week_cos'] = np.cos(2 * np.pi * df['week_of_year'] / 52)

# Indicadores especiales
df['is_q4'] = (df['quarter'] == 4).astype(int)  # Q4 suele tener alta demanda
df['is_january'] = (df['month'] == 1).astype(int)  # Enero post-navidad

print("✓ Features temporales creadas")
print(f"  • Columnas temporales: {['year', 'month', 'quarter', 'week_of_year', 'month_sin', 'month_cos', 'week_sin', 'week_cos']}")

## 3. Lag Features (Rezagos)

In [ ]:
# Crear lags de ventas por producto
lag_periods = [1, 2, 4, 8, 52]  # 1w, 2w, 1m, 2m, 1y

for lag in lag_periods:
    df[f'sales_lag_{lag}'] = df.groupby('item_id')['total_sales'].shift(lag)
    print(f"  • Lag {lag} semanas creado")

# Lags de urgencias (binario)
for lag in [1, 2, 4]:
    df[f'urgent_lag_{lag}'] = df.groupby('item_id')['is_urgent_a'].shift(lag)

print("\n✓ Lag features creadas")

## 4. Rolling Statistics (Ventanas Móviles)

In [ ]:
# Media móvil y estadísticas en ventanas de 4, 8, 12 semanas
windows = [4, 8, 12]

for window in windows:
    # Media móvil
    df[f'ma_{window}'] = df.groupby('item_id')['total_sales'].transform(
        lambda x: x.rolling(window=window, min_periods=1).mean()
    )
    
    # Desviación estándar móvil
    df[f'std_{window}'] = df.groupby('item_id')['total_sales'].transform(
        lambda x: x.rolling(window=window, min_periods=1).std()
    )
    
    # Mínimo y máximo móvil
    df[f'min_{window}'] = df.groupby('item_id')['total_sales'].transform(
        lambda x: x.rolling(window=window, min_periods=1).min()
    )
    
    df[f'max_{window}'] = df.groupby('item_id')['total_sales'].transform(
        lambda x: x.rolling(window=window, min_periods=1).max()
    )
    
    print(f"  • Rolling stats ventana {window} semanas")

# Features derivadas
df['sales_vs_ma4'] = df['total_sales'] / (df['ma_4'] + 1)  # Ratio vs tendencia
df['sales_vs_ma12'] = df['total_sales'] / (df['ma_12'] + 1)

print("\n✓ Rolling statistics creadas")

## 5. Features Específicas de Urgencias

In [ ]:
def calcular_dias_desde_ultima_urgencia(grupo):
    """Calcula días desde la última urgencia para cada producto"""
    dias = []
    ultimo_urgente = None
    
    for idx, row in grupo.iterrows():
        if ultimo_urgente is None:
            dias.append(0)
        else:
            dias.append((row['date'] - ultimo_urgente).days)
        
        if row['is_urgent_a'] == 1:
            ultimo_urgente = row['date']
    
    return pd.Series(dias, index=grupo.index)

# Días desde última urgencia
df['days_since_urgent'] = df.groupby('item_id').apply(calcular_dias_desde_ultima_urgencia).values

# Frecuencia de urgencias en ventanas móviles
for window in [4, 8, 12]:
    df[f'urgent_freq_{window}w'] = df.groupby('item_id')['is_urgent_a'].transform(
        lambda x: x.rolling(window=window, min_periods=1).sum()
    )
    
    df[f'urgent_rate_{window}w'] = df[f'urgent_freq_{window}w'] / window

print("✓ Features de urgencias creadas")
print(f"  • days_since_urgent: promedio {df['days_since_urgent'].mean():.1f} días")
print(f"  • urgent_rate_4w: promedio {df['urgent_rate_4w'].mean():.2%}")

## 6. Features de Tendencia

In [ ]:
# Diferencias (cambio respecto a semana anterior)
df['sales_diff_1'] = df.groupby('item_id')['total_sales'].diff(1)
df['sales_diff_4'] = df.groupby('item_id')['total_sales'].diff(4)

# Tasa de crecimiento
df['sales_pct_change_1'] = df.groupby('item_id')['total_sales'].pct_change(1)
df['sales_pct_change_4'] = df.groupby('item_id')['total_sales'].pct_change(4)

# Momentum (diferencia entre MA corta y larga)
df['momentum'] = df['ma_4'] - df['ma_12']

print("✓ Features de tendencia creadas")

## 7. Limpieza y Preparación Final

In [ ]:
# Revisar valores faltantes
print("Missing values por columna:")
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
print(missing.head(10))

# Rellenar missing values
# Para lags: forward fill (usamos el valor anterior)
lag_cols = [col for col in df.columns if 'lag_' in col or 'std_' in col]
df[lag_cols] = df.groupby('item_id')[lag_cols].fillna(method='ffill')

# Para diferencias y pct_change: rellenar con 0
diff_cols = [col for col in df.columns if 'diff' in col or 'pct_change' in col]
df[diff_cols] = df[diff_cols].fillna(0)

# Reemplazar infinitos por NaN y luego rellenar
df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)

print(f"\n✓ Missing values manejados")
print(f"  • Nulls restantes: {df.isnull().sum().sum()}")

## 8. Análisis de Correlaciones

In [ ]:
# Seleccionar features numéricas relevantes
feature_cols = [col for col in df.columns if col not in [
    'item_id', 'date', 'wm_yr_wk', 'total_sales', 'is_urgent_a', 'is_urgent_b', 'is_urgent_c',
    'threshold_a', 'ma_4_b', 'threshold_c', 'mean_prod', 'std_prod'
]]

# Calcular correlación con variable objetivo
correlations = df[feature_cols + ['is_urgent_a']].corr()['is_urgent_a'].drop('is_urgent_a')
correlations = correlations.sort_values(ascending=False)

print("\n📊 TOP 15 FEATURES MÁS CORRELACIONADAS CON URGENCIA:")
print(correlations.head(15))

print("\n📊 BOTTOM 10 FEATURES (menor correlación):")
print(correlations.tail(10))

In [ ]:
# Visualizar top correlaciones
fig, ax = plt.subplots(figsize=(10, 8))
top_corr = correlations.head(20)
top_corr.plot(kind='barh', ax=ax, color='steelblue')
ax.set_xlabel('Correlación con is_urgent_a')
ax.set_title('Top 20 Features por Correlación con Urgencias')
ax.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.tight_layout()
plt.savefig('../results/feature_correlations.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Gráfico guardado: results/feature_correlations.png")

## 9. Train/Test Split Temporal

In [ ]:
# Split temporal: últimas 26 semanas (6 meses) para test
fechas_unicas = sorted(df['date'].unique())
fecha_split = fechas_unicas[-26]  # Últimas 26 semanas para test

df_train = df[df['date'] < fecha_split].copy()
df_test = df[df['date'] >= fecha_split].copy()

print(f"\n📅 SPLIT TEMPORAL:")
print(f"\nTRAIN:")
print(f"  • Fechas: {df_train['date'].min()} a {df_train['date'].max()}")
print(f"  • Semanas: {df_train['date'].nunique()}")
print(f"  • Registros: {len(df_train):,}")
print(f"  • Urgencias: {df_train['is_urgent_a'].sum():,} ({df_train['is_urgent_a'].mean():.1%})")

print(f"\nTEST:")
print(f"  • Fechas: {df_test['date'].min()} a {df_test['date'].max()}")
print(f"  • Semanas: {df_test['date'].nunique()}")
print(f"  • Registros: {len(df_test):,}")
print(f"  • Urgencias: {df_test['is_urgent_a'].sum():,} ({df_test['is_urgent_a'].mean():.1%})")

## 10. Guardar Datasets Preparados

In [ ]:
# Guardar dataset completo con features
df.to_csv('../data/simulated/dataset_features.csv', index=False)
print(f"✓ Dataset completo guardado: data/simulated/dataset_features.csv")
print(f"  • Shape: {df.shape}")

# Guardar train/test
df_train.to_csv('../data/simulated/train_features.csv', index=False)
df_test.to_csv('../data/simulated/test_features.csv', index=False)
print(f"\n✓ Train/Test guardados:")
print(f"  • train_features.csv: {df_train.shape}")
print(f"  • test_features.csv: {df_test.shape}")

# Guardar lista de features para modelado
features_modeling = [col for col in feature_cols if col in df.columns]
pd.DataFrame({'feature': features_modeling}).to_csv('../data/simulated/feature_list.csv', index=False)
print(f"\n✓ Lista de features guardada: {len(features_modeling)} features")

## 11. Resumen Final

In [ ]:
print("\n" + "="*70)
print("📊 RESUMEN FEATURE ENGINEERING")
print("="*70)

print(f"\n✓ Productos procesados: {df['item_id'].nunique():,}")
print(f"✓ Registros totales: {len(df):,}")
print(f"✓ Features creadas: {len(feature_cols)}")

print(f"\n📂 ARCHIVOS GENERADOS:")
print(f"  • dataset_features.csv - Dataset completo con todas las features")
print(f"  • train_features.csv - Training set ({len(df_train):,} registros)")
print(f"  • test_features.csv - Test set ({len(df_test):,} registros)")
print(f"  • feature_list.csv - Lista de features para modelado")
print(f"  • feature_correlations.png - Visualización de correlaciones")

print(f"\n🎯 CARACTERÍSTICAS DEL DATASET:")
print(f"  • Target: is_urgent_a (Definición A: Percentil 75)")
print(f"  • Balance: {df['is_urgent_a'].mean():.1%} urgencias")
print(f"  • Horizonte temporal: {df['date'].nunique()} semanas")
print(f"  • Split: {len(df_train)} train / {len(df_test)} test")

print(f"\n✅ DATASET LISTO PARA MODELADO")
print(f"\nPróximos pasos:")
print(f"  → Fase 4: Modelado con Prophet")
print(f"  → Fase 5: Modelado con XGBoost")
print(f"  → Fase 6: Modelado con Random Forest")
print(f"  → Fase 7: Comparación de modelos")
print("="*70)